# Computing volume fractions by intersecting unstructured meshes

**Code used to generate the data**: 

**Article DOI**: 

**Research data archive**:

## Surface-Mesh / Cell Intersection Algorithm

**TODO**: Short description


## Sphere Initialization

In [ ]:
# Pick up .csv files 
import os
import pandas as pd
from math import pi
from plot_study import *

In [ ]:
import warnings 
# Set the GEOM_VOF_INIT environmental variable to the folder where you want the data to be saved. 
data_dir = os.curdir
try:
    data_dir = os.environ["GEOM_VOF_INIT"] 
except:
    warnings.warn("No GEOM_VOF_INIT environmental variable set, storing data in the current working directory.") 

In [ ]:
# Exact volume of the sphere.  
sphere_radius = 0.15 # WARNING: Adapt this if another radius is chosen for the sphere in the study. 
sphere_volume = sphere_radius** 3 * 4 * pi / 3.

plot_study("sphere", "blockMesh", alg_name="SMCI", exact_volume=sphere_volume, data_dir=data_dir)

In [ ]:
plot_study("sphere", "blockMesh", alg_name="POLY", exact_volume=sphere_volume, data_dir=data_dir, csv_file="polynomialVofInitResults.csv")